In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install "unsloth[colab_ampere] @ git+https://github.com/unslothai/unsloth.git"
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
pass

!pip install "git+https://github.com/huggingface/transformers.git" # Native 4bit loading works!

In [ ]:
# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import os
import torch
from datasets import load_dataset

from unsloth import FastLanguageModel
from peft import LoraConfig
from trl import SFTTrainer
import transformers

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:72: UserWarning: Unsloth: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


## Restart your session

In [ ]:
dtype = None
max_seq_length=2048
# Load the base model
model,tokenizer = FastLanguageModel.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    device_map="auto",
    max_seq_length = max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)


Unsloth: You passed in `mistralai/Mistral-7B-v0.1` and `load_in_4bit = True`.
We shall load `unsloth/mistral-7b-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# SECTION 1.2
## Setting up Dataset
- The Llama-chat model expects prompts of the form:
[INST] {user_message} [/INST]

To train the model, we construct our training data like this:
[INST] {user_message} [/INST] {response}

In [ ]:
from datasets import load_dataset

dataset_name = 'sid6i7/patient-doctor'
dataset = load_dataset(dataset_name, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def formatting_prompts_func(batch):
    texts = [value.replace("<start>", "").replace("<end>", "").strip() for value in batch["Doctor"]]
    return {"Doctor": texts}

In [ ]:
dataset = dataset.map(formatting_prompts_func,batched=True)

Map:   0%|          | 0/106220 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['Unnamed: 0', 'Description', 'Patient', 'Doctor', 'problem_description'],
    num_rows: 106220
})

In [ ]:
from IPython.display import Markdown
Markdown(dataset["Description"][0])

How can one treat intense pain in the stomach while suffering from ulcer?

In [ ]:
Markdown(dataset["Doctor"][0])


Hello,I don't think your ulcer is coming back but you should be absolutely careful with the diet as your doctor has advised you. If it will be very intense than contact your doctor and he or she will advise you further about it.Hope I have answered your query. Let me know if I can assist you further.
 Regards,
Dr. Jolanda Nikolla, Pulmonologist

In [ ]:
alpaca_prompt = """Below is an input that describes a task and provides further context. Write a response that appropriately completes the request.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["Description"]
    responses      = examples["Doctor"]
    texts = []
    for instruction, input in zip(instructions, responses):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)


dataset = dataset.map(formatting_prompts_func, remove_columns=[f for f in dataset.features if not f == 'text'],batched=True)

Map:   0%|          | 0/106220 [00:00<?, ? examples/s]

In [ ]:
Markdown(dataset["text"][0])

Below is an input that describes a task and provides further context. Write a response that appropriately completes the request.

### Input:
How can one treat intense pain in the stomach while suffering from ulcer?

### Response:
Hello,I don't think your ulcer is coming back but you should be absolutely careful with the diet as your doctor has advised you. If it will be very intense than contact your doctor and he or she will advise you further about it.Hope I have answered your query. Let me know if I can assist you further.
 Regards,
Dr. Jolanda Nikolla, Pulmonologist</s>

In [ ]:
new_model = 'mistral-7b-medical-assistance'

# SECTION 1.3 Training with LORA

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=4):   0%|          | 0/106220 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 106,220 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 2
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.474200
2,2.699900
3,2.597000
4,2.345400
5,2.265900
6,2.010300
7,1.977200
8,2.078100
9,2.050600
10,1.980400


In [ ]:
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Suggest remedies for me. I am currently suffering from acute pains in my throat.", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 200 ) # optionally set temperature and do_sample=True
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an input that describes a task and provides further context. Write a response that appropriately completes the request.\n\n### Input:\nSuggest remedies for me. I am currently suffering from acute pains in my throat.\n\n### Response:\nHello,\n I have gone through your query.\n I understand your concern.\n I would suggest you to take steam inhalation with hot water and salt.\n Take hot water with honey and lemon juice.\n Take hot water with turmeric powder.\n Take hot water with ginger juice.\n Take hot water with black pepper powder.\n Take hot water with cinnamon powder.\n Take hot water with clove powder.\n Take hot water with cardamom powder.\n Take hot water with mint leaves.\n Take hot water with basil leaves.\n Take hot water with tulsi leaves.\n Take hot water with fenugreek seeds.\n Take hot water with coriander seeds.\n Take hot water with cumin seeds.\n Take hot water with fennel seeds.\n Take hot water with anise seeds.\n Take hot water with caraway seeds.\n Ta

In [ ]:
model.save_pretrained_merged(new_model, tokenizer, save_method = "merged_16bit")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.41 out of 12.67 RAM for saving.


100%|██████████| 32/32 [00:31<00:00,  1.02it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving mistral-7b-medical-assistance/pytorch_model-00001-of-00003.bin...
Unsloth: Saving mistral-7b-medical-assistance/pytorch_model-00002-of-00003.bin...
Unsloth: Saving mistral-7b-medical-assistance/pytorch_model-00003-of-00003.bin...
Done.


In [ ]:
model.push_to_hub_merged(new_model, tokenizer, save_method = "merged_16bit")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.57 out of 12.67 RAM for saving.


100%|██████████| 32/32 [00:39<00:00,  1.22s/it]


Unsloth: Saving tokenizer...

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-660d6793-07cfaafb4269f03e64a8ecec;8ef1edbd-3a42-4d07-95f8-7fc0a94b65b6)

Repository Not Found for url: https://huggingface.co/api/models/mistral-7b-medical-assistance.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.